In [49]:
from affinewarp import PiecewiseWarping, SpikeData
import numpy as np
from affinewarp.visualization import rasters    
import matplotlib.pyplot as plt
from DataPreprocessing import *
from DataPreprocessing import *
import elephant.conversion as conv
import neo
import quantities as pq
from elephant.statistics import instantaneous_rate,time_histogram
import neo
from elephant import kernels
from quantities import Hz, s, ms


In [4]:
mid_brain_circuits=['SCs','SCm','MRN','APN','PAG','ZI']
frontal_circuits=['MOs','PL','ILA','ORB','MOp','SSp']
all_data_path='/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/unzipped_files'

In [30]:
#Load data

data_path= '/media/maria/DATA1/Documents/NeuroMatchAcademy2020_dat/unzipped_files/Richards_2017-10-31.tar'

trials_intervals = np.load(data_path+'/'+'trials.intervals.npy') # in seconds
spike_times = np.load(data_path+'/'+'spikes.times.npy') * 1000 # Unbinned spike times in ms
trials_gocue_times = np.load(data_path+'/'+'trials.goCue_times.npy') 
trials_response_choice = np.load(data_path+'/'+'trials.response_choice.npy') # -1 left, 1, right, 0 no response
spontaneous_intervals = np.load(data_path+'/'+'spontaneous.intervals.npy')
trials_response_time = np.load(data_path+'/'+'trials.response_times.npy')
spike_clusters = np.load(data_path+'/'+'spikes.clusters.npy')
site_positions = np.load(data_path+'/'+'channels.sitePositions.npy')
clusters_depths = np.load(data_path+'/'+'clusters.depths.npy')
clusters_annotation = np.load(data_path+'/'+'clusters._phy_annotation.npy')
channel_sites = np.load(data_path+'/'+'channels.site.npy')
channels_brainlocation = pd.read_csv(data_path+'/'+'channels.brainLocation.tsv', sep='\t')
clusters_probes = np.load(data_path+'/'+'clusters.probes.npy')
channels_probe = np.load(data_path+'/'+'channels.probe.npy')
trials_visual_time = np.load(data_path+'/'+'trials.visualStim_times.npy')

good_cells, brain_regions,br = get_good_cells(data_path) #
#print(good_cells)

def get_neurons(spike_times,spike_clusters):
    spike_time_cells = np.empty(len(np.unique(spike_clusters)), dtype=object) # Initalise empty object
    for i in range(len(np.unique(spike_clusters))):
      # Create a spike time arrays, where each array in the array is a spike time of a cell
      spike_time_cells[i] = spike_times[(np.where(spike_clusters == i)[0])]
    return spike_time_cells

def make_arrays(spike_time_cells,trials_intervals):
    spikes=[]
    neurons=[]
    trials=[]
    trials_orig=trials_intervals*1000  
    neurons_orig=range(len(spike_time_cells))
    
    for neuron in neurons_orig:
        #spk_ids=spike_time_cells[neuron]
        spk_tms_one_neuron=spike_time_cells[neuron]
        for trial in range(0,260):
            trial_range= np.bitwise_and(spk_tms_one_neuron>=trials_orig[trial][0],spk_tms_one_neuron<=trials_orig[trial][1])
            if trial==0:
                #trial_range= np.bitwise_and(spk_tms_one_neuron>=trials[trial][0],spk_tms_one_neuron<=trials[trial][1])
                subset=spk_tms_one_neuron[trial_range]
            else:
                subset=spk_tms_one_neuron[trial_range]-trials_orig[trial-1][1]
            for spike in subset:
                trials.append(trial)
                spikes.append(spike)
                neurons.append(neuron)
    return neurons,trials,spikes

def run_affinewarp(neurons,trials,spikes):
    data = SpikeData(trials, spikes, neurons, tmin=0, tmax=1500)
    binned = data.bin_spikes(n_bins=20)
    from affinewarp import PiecewiseWarping
    model = PiecewiseWarping()
    model.fit(binned)
    aligned_data = model.transform(data)
    return aligned_data
    
    
def preprocess_for_affine_warp(bin_size):
    good_cells, brain_regions,br = get_good_cells(data_path) # Get brain regions
    spike_time_cells=get_neurons(spike_times,spike_clusters)
    #neurons,trials,spikes=make_arrays(spike_time_cells,trials_intervals)
    #neuron_trials=extract_trial_aligned(spike_time_cells,trials_intervals)
    neurons=np.load('neurons.npy')
    trials=np.load('trials.npy')
    spikes=np.load('spikes.npy')
    aligned_data=run_affinewarp(neurons,trials,spikes)
    #mean_lst=bin_and_average(neuron_trials,trials_intervals,bin_size=10)
    #dat_for_affine_warp= preprocess_for_affine_warp(neuron_trials,trials_intervals,bin_size=10)
    #spike_time_binned, spike_time_cells, cell_spikes_max = bin_spikes(spike_times,spike_clusters,clusters_annotation,bin_size=bs)
    #return neurons,trials,spikes
    return aligned_data
aligned_data=preprocess_for_affine_warp(10)


Loss improvement: 5.83%: 100%|██████████| 50/50 [01:11<00:00,  1.43s/it]


In [39]:
print(len(aligned_data.trials))

1553247


In [55]:
trials=aligned_data.trials
trials[trials==259]

array([259, 259, 259, ..., 259, 259, 259])

In [52]:
def convert_from_aligned_data(aligned_data):
    neurons=aligned_data.neurons
    trials=aligned_data.trials
    spikes=aligned_data.spiketimes
    #mean_trials=[]
    #trials_intervals=trials_intervals*1000
    for neuron in range(0,np.unique(neurons).shape[0]):
        one_neuron_trial=np.where(trials[neurons==neuron])[0]
        one_neuron_spikes=spikes[one_neuron_trial]
        #print(one_neuron_spikes)
        if one_neuron_spikes!=[]:
            start=np.min(one_neuron_spikes)
            end=np.max(one_neuron_spikes)
            spk_tr=neo.SpikeTrain(one_neuron_spikes*pq.ms,t_start=start*pq.ms,t_stop=end*pq.ms)
            binned_spk_tr = conv.BinnedSpikeTrain(spk_tr, binsize=0.02524578*pq.s,
                                    t_start=start*pq.ms,t_stop=end*pq.ms)
            binned_spk_tr=binned_spk_tr.to_array()
            print(binned_spk_tr)
        
        '''
        for trial in range(260):
            
            spk_tr=neo.SpikeTrain(neuron_trials[neuron][trial]*pq.ms,t_start=start*pq.ms,t_stop=end*pq.ms)
            binned_spk_tr = conv.BinnedSpikeTrain(spk_tr, binsize=0.02524578*pq.s,
                                t_start=start*pq.ms,t_stop=end*pq.ms)
            binned_spk_tr=binned_spk_tr.to_array()
            print(binned_spk_tr.shape)
            neuron_av.append(binned_spk_tr)
        #neuron_av.append(np.mean(binned_spk_tr,axis=0))
        mean_trials.append(neuron_av)
        
    minim=[]
    for j in range(0,len(neuron_trials.keys())):
        for i in range(0,trials_intervals.shape[0]):
            minim.append(mean_lst[j][i].shape[1])
    minimum=np.min(minim)

    av_tot=[]
    for j in range(0,len(neuron_trials.keys())):
        av=[]
        for i in range(0,trials_intervals.shape[0]):
            av.append(mean_lst[j][i].flatten()[:minim])
            #print(av[-1].shape)
        av=np.array(av)
        print(av.shape)
        av_mean=np.mean(av,axis=0)
        av_tot.append(av_mean)
    av_tot=np.array(av_tot)
            
    return av_tot
    '''
convert_from_aligned_data(aligned_data)

/home/maria/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:11: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  # This is added back by InteractiveShellApp.init_path()


[[600]]
[[1373]]
[[295]]
[[221]]
[[635]]
[[190]]
[[249]]
[[71]]
[[183]]
[[17]]
[[241]]
[[1221]]
[[489]]
[[195]]
[[216]]
[[707]]
[[227]]
[[38]]
[[389]]
[[218]]
[[337]]
[[1798]]
[[375]]
[[544]]
[[49]]
[[84]]
[[105]]
[[3500]]
[[868]]
[[179]]
[[22]]
[[64]]
[[117]]
[[1605]]
[[158]]
[[6]]
[[130]]
[[73]]
[[2205]]
[[622]]
[[986]]
[[47]]
[[151]]
[[542]]
[[1288]]
[[591]]
[[27]]
[[367]]
[[2023]]
[[169]]
[[387]]
[[314]]
[[182]]
[[94]]
[[20]]
[[110]]
[[36]]
[[57]]
[[1518]]
[[596]]
[[212]]
[[16]]
[[1578]]
[[1289]]
[[133]]
[[31]]
[[427]]
[[276]]
[[93]]
[[18]]
[[225]]
[[487]]
[[31]]
[[145]]
[[1101]]
[[645]]
[[1483]]
[[981]]
[[621]]
[[59]]
[[392]]
[[25]]
[[1337]]
[[877]]
[[140]]
[[450]]
[[851]]
[[839]]
[[1210]]
[[52]]
[[1526]]
[[40]]
[[83]]
[[158]]
[[389]]
[[1291]]
[[613]]
[[11]]
[[70]]
[[20]]
[[1094]]
[[289]]
[[344]]
[[320]]
[[1152]]
[[31]]
[[525]]
[[79]]
[[47]]
[[9]]
[[103]]
[[354]]
[[848]]
[[158]]
[[286]]
[[46]]
[[285]]
[[319]]
[[354]]
[[100]]
[[3777]]
[[185]]
[[230]]
[[128]]
[[986]]
[[750]]
[[419]]

ValueError: Starting time of each spike train must be smaller than each stopping time